# User Item Biases With Regularization
* Prediction for user $i$ and item $j$ is $\tilde r_{ij} = u_i + a_j$
* Loss function is $L = \sum_{\Omega}w_{ij}(r_{ij} - u_i - a_j)^2 + \lambda_u \sum_i (u_i - \bar u) ^2 + \lambda_a \sum_j (a_j - \bar a)^2 $
* $\bar u$ is the mean of $u_i$ and $\bar a$ is the mean of $a_j$ 
* $\Omega$ is the set of oberved pairs $(i, j)$
* $r_{ij}$ is the rating for user $i$ and item $j$
* $w_{ij}$ is the weight for the prediction $r_{ij}$ and is modeled as a power-law in the number of items seen by $i$ and users than have seen $j$: $w_{ij} = |j' : (i, j') \in \Omega| ^ {\lambda_{wu}} |i' : (i', j) \in \Omega| ^ {\lambda_{wa}}$

In [1]:
const name = "UserItemBiases"
const residual_alphas = []
const validation_weight_scheme = "inverse";

In [2]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [3]:
const training = get_residuals("training", residual_alphas)
const validation = get_residuals("validation", residual_alphas);

## Alternating Least Squares Algorithm
* $u_i = \dfrac{\sum_{j \in \Omega_i}(r_{ij} - a_j) w_{ij} + \bar u \lambda_u}{ \sum_{j \in \Omega_i} w_{ij} + \lambda_u}$
* $\Omega$ is the set of (user, item) pairs that we have ratings for
* $\Omega_i$ is subset of $\Omega$ for which the user is the $i$-th user

In [4]:
function get_residuals!(users, items, ratings, weights, a, ρ, Ω)
    for row = 1:length(users)
        i = users[row]
        j = items[row]
        r = ratings[row]
        w = weights[row]
        ρ[i] += (r - a[j]) * w
        Ω[i] += w
    end
    ρ, Ω
end

function thread_range(n)
    tid = Threads.threadid()
    nt = Threads.nthreads()
    d, r = divrem(n, nt)
    from = (tid - 1) * d + min(r, tid - 1) + 1
    to = from + d - 1 + (tid ≤ r ? 1 : 0)
    from:to
end

function update_users!(users, items, ratings, weights, u, a, λ_u, ρ, Ω)
    Threads.@threads for t = 1:Threads.nthreads()
        range = thread_range(length(ratings))
        ρ[:, Threads.threadid()] .= 0
        Ω[:, Threads.threadid()] .= 0
        @views get_residuals!(
            users[range],
            items[range],
            ratings[range],
            weights[range],
            a,
            ρ[:, Threads.threadid()],
            Ω[:, Threads.threadid()],
        )
    end

    ρ = sum(ρ, dims = 2)
    Ω = sum(Ω, dims = 2)

    μ = mean(u)
    Threads.@threads for i = 1:length(u)
        u[i] = (ρ[i] + μ * λ_u) / (Ω[i] + λ_u)
    end
end;

In [5]:
function train_model(training, stop_criteria, λ_u, λ_a, λ_wu, λ_wa)
    @info "training model with parameters [$λ_u, $λ_a, $λ_wu, $λ_wa]"
    users, items, ratings = training.user, training.item, training.rating
    weights =
        safe_exp.(get_counts("training"), log(λ_wu)) .*
        safe_exp.(get_counts("training"; by_item = true), log(λ_wa))
    u = zeros(eltype(λ_u), maximum(users))
    a = zeros(eltype(λ_a), maximum(items))

    ρ_u = zeros(eltype(u), length(u), Threads.nthreads())
    Ω_u = zeros(eltype(u), length(u), Threads.nthreads())
    ρ_a = zeros(eltype(a), length(a), Threads.nthreads())
    Ω_a = zeros(eltype(a), length(a), Threads.nthreads())

    while !stop!(stop_criteria, [u, a])
        update_users!(users, items, ratings, weights, u, a, λ_u, ρ_u, Ω_u)
        update_users!(items, users, ratings, weights, a, u, λ_a, ρ_a, Ω_a)
    end
    u, a
end;

In [6]:
function make_prediction(users, items, u, a)
    r = zeros(eltype(u), length(users))
    u_mean = mean(u)
    a_mean = mean(a)
    for i = 1:length(r)
        if users[i] > length(u)
            r[i] += mean(u)
        else
            r[i] += u[users[i]]
        end
        if items[i] > length(a)
            r[i] += mean(a)
        else
            r[i] += a[items[i]]
        end
    end
    r
end;

## Training

In [7]:
function validation_mse(λ)
    λ = exp.(λ) # ensure λ is nonnegative
    stop_criteria = convergence_stopper(1e-6, max_iters = 16)
    u, a = train_model(training, stop_criteria, λ...)
    pred_score = make_prediction(validation.user, validation.item, u, a)
    weights = get_weights("validation", validation_weight_scheme)
    mse(validation.rating, pred_score, weights)
end;

In [8]:
# Find the best regularization hyperparameters
res = optimize(
    validation_mse,
    fill(0.0f0, 4),
    LBFGS(),
    autodiff = :forward,
    Optim.Options(show_trace = true, extended_trace = true),
);
λ = exp.(Optim.minimizer(res));

[ Info: 20220514 02:02:32 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,1.0,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,1.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,0.0,1.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,0.0,0.0,1.0)]
Progress: 100%|███████████████████████████| Time: 0:00:01 ( 0.12 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (35.72 ns/it)


Iter     Function value   Gradient norm 
     0     1.820281e+00     5.464972e-02
 * Current step size: 1.0
 * time: 0.02705097198486328
 * g(x): Float32[-0.0050086514, -1.0955863f-6, 0.019451264, 0.05464972]
 * x: Float32[0.0, 0.0, 0.0, 0.0]


[ Info: 20220514 02:03:02 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0050212,1.0050212,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000011,0.0,1.0000011,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9807367,0.0,0.0,0.9807367,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.94681674,0.0,0.0,0.0,0.94681674)]
Progress: 100%|███████████████████████████| Time: 0:00:00 (36.67 ns/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (36.00 ns/it)
[ Info: 20220514 02:03:49 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.004674,1.004674,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000011,0.0,1.0000011,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.98205394,0.0,0.0,0.98205394,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9503939,0.0,0.0,0.0,0.9503939)]
Progress: 100%|███████████

     1     1.818468e+00     1.585315e-02
 * Current step size: 0.930997
 * time: 96.36286497116089
 * g(x): Float32[-0.0008187882, -1.7059443f-6, 0.015853152, -0.003936903]
 * x: Float32[0.0046630395, 1.0199876f-6, -0.018109068, -0.05087873]


[ Info: 20220514 02:04:36 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0057998,1.0057998,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000026,0.0,1.0000026,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9665626,0.0,0.0,0.9665626,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9499285,0.0,0.0,0.0,0.9499285)]
Progress: 100%|███████████████████████████| Time: 0:00:00 (36.63 ns/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (35.95 ns/it)
[ Info: 20220514 02:05:23 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0103159,1.0103159,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000092,0.0,1.0000092,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9070028,0.0,0.0,0.9070028,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9480692,0.0,0.0,0.0,0.9480692)]
Progress: 100%|█████████████

...

    18     1.814453e+00     1.540225e-05
 * Current step size: 4.7776413
 * time: 2640.0163099765778
 * g(x): Float32[-1.6312562f-6, 6.459795f-10, 7.660067f-6, 1.540225f-5]
 * x: Float32[-0.35804656, -8.422065, -0.44261852, -0.017399423]


[ Info: 20220514 02:47:00 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.69904435,0.69904435,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.00017783235,0.0,0.00017783235,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.64232135,0.0,0.0,0.64232135,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.98275584,0.0,0.0,0.0,0.98275584)]
Progress: 100%|███████████████████████████| Time: 0:00:00 (36.58 ns/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (36.02 ns/it)
[ Info: 20220514 02:47:47 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.6990597,0.6990597,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(7.597643e-5,0.0,7.597643e-5,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.642198,0.0,0.0,0.642198,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.98277503,0.0,0.0,0.0,0.98277503)]
Progress: 

    19     1.814453e+00     4.696696e-06
 * Current step size: 1.6080762
 * time: 2780.8111758232117
 * g(x): Float32[4.7711893f-7, 3.972438f-10, -2.147419f-6, -4.6966957f-6]
 * x: Float32[-0.35803774, -8.763949, -0.4426957, -0.017391577]


[ Info: 20220514 02:49:21 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.69903857,0.69903857,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.00012561135,0.0,0.00012561135,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.64230627,0.0,0.0,0.64230627,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9827574,0.0,0.0,0.0,0.9827574)]
Progress: 100%|███████████████████████████| Time: 0:00:00 (36.61 ns/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (36.02 ns/it)
[ Info: 20220514 02:50:08 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.69900584,0.69900584,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(5.2442563e-5,0.0,5.2442563e-5,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.64232075,0.0,0.0,0.64232075,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.98275185,0.0,0.0,0.0,0.98275185)]
Prog

    20     1.814453e+00     7.348267e-06
 * Current step size: 6.887216
 * time: 2968.514011859894
 * g(x): Float32[-8.7457266f-7, 6.163955f-11, 3.4997806f-6, 7.3482665f-6]
 * x: Float32[-0.35811824, -10.2679, -0.44265687, -0.017401274]


In [9]:
@info "The optimal [λ_u, λ_a, λ_w] is $λ, found in " *
      repr(Optim.f_calls(res)) *
      " function calls"

[ Info: 20220514 02:52:28 The optimal [λ_u, λ_a, λ_w] is Float32[0.6989904, 3.4730216f-5, 0.64232755, 0.9827492], found in 64 function calls


In [10]:
empty!(memoize_cache(get_weights))
stop_criteria = convergence_stopper(1e-6, max_iters = 16)
u, a = train_model(training, stop_criteria, λ...);

[ Info: 20220514 02:52:29 training model with parameters [0.6989904, 3.4730216e-5, 0.64232755, 0.9827492]
Progress: 100%|███████████████████████████| Time: 0:00:00 (36.58 ns/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (35.95 ns/it)


## Inference

In [11]:
model(users, items) = make_prediction(users, items, u, a);

In [12]:
write_predictions(model; residual_alphas = residual_alphas);

[ Info: 20220514 02:53:10 training set: RMSE 1.2844313 MAE 0.960797 R2 0.46220547
Progress: 100%|███████████████████████████| Time: 0:00:00 (36.53 ns/it)
[ Info: 20220514 02:53:18 training set weighted-loss: RMSE 1.2922145 MAE 0.9602112 R2 0.48654854
[ Info: 20220514 02:53:18 validation set: RMSE 1.3195661 MAE 0.9857658 R2 0.40952367
[ Info: 20220514 02:53:19 validation set weighted-loss: RMSE 1.3469539 MAE 1.0024743 R2 0.39468563


In [13]:
write_params(Dict("u" => u, "a" => a, "λ" => λ));